# Example of using QuOCS with Qudi

We want to run an optimization on one of our test-problems provided in QuOCS. TO demonstrate some of the advantages of QuOCS in experiment optimization, we add noise to the figure of merit (FoM) and make use of the re-evaluation steps functionality (see below). But let's go through the setup of the optimization step-py-step.

## Import statements

We need to import the QuOCS Optimizer and the test-problem (OneQubitProblem). Let's also include numpy for good measure becuase we might need it later.

In [2]:
from quocslib.Optimizer import Optimizer
# from quocslib.optimalcontrolproblems.OneQubitProblem_2fields import OneQubit2Fields
from quocslib.optimalcontrolproblems.OneQubitProblem import OneQubit
import numpy as np

## The Configuration of QuOCS

To configure QuOCS, you need to fill the ```optimization_dictionary``` with information. In this example, we will do it directly in the code, but you can also load the dictionary from a JSON file.

Let's define the name of the optimization. This will also show up in the name of the folder containing the results.

In [3]:
optimization_client_name = "dCRAB_Noisy_2_control_fields"

optimization_dictionary = {"optimization_client_name": optimization_client_name}

Now we need to specify the general settings, such as the used algorithm (in this case dCRAB) and its parameters (e.g. number of super-iterations). In the same section we choos to use the re-evaluation steps method for noisy optimization. Extending the dCRAB algorithm, the direct dearch method (dsm) needs to be specified and stopping criteria for each sub-iteration can be defined.

In [4]:
optimization_dictionary["algorithm_settings"] = {
        "algorithm_name": "dCRAB",
        "super_iteration_number": 5,
        "max_eval_total": 5000,
        "re_evaluation": {
            "re_evaluation_steps": [0.3, 0.5, 0.51]
        },
        "dsm_settings": {
            "general_settings": {
                "dsm_algorithm_name": "NelderMead",
                "is_adaptive": true
            },
            "stopping_criteria": {
                "xatol": 1e-2,
                "frtol": 1e-2,
                "max_eval": 200
            }
        }
    }

### Times

The time of a pulse is specified separately to use it as an optimization parameter in future updates and so that different pulses can act on separate timescales but can be grouped by common durations. Since there can me more than one time, the entries have to be provided as a list (note the square brackets around the dictionary for ```time_1```):

In [5]:
optimization_dictionary["times"] = [{
        "time_name": "time_1",
        "initial_value": 3.0
    }]

### Parameters

Since we do not have any parameters in this example, we just add an empty list for the settings of parameter optimization.

In [6]:
optimization_dictionary['parameters'] = []

### Pulses

In [10]:
pulse_amplitude = {'pulse_name': 'Amplitude', 
                   'upper_limit': 15.0, 
                   'lower_limit': 0.1, 
                   'bins_number': 101, 
                   'time_name': 'time_p', 
                   'amplitude_variation': 0.5, 
                   'basis': {'basis_name': 'Fourier', 
                             'basis_class': 'Fourier', 
                             'basis_module': 'quocslib.pulses.basis.Fourier', 
                             'basis_vector_number': 5, 
                             'random_super_parameter_distribution': 
                             {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                                   'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                                   'lower_limit': 0.1, 'upper_limit': 3.0}
                            }, 
                   'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
                   'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: np.pi/3 + 0.0*t'}
                  }

In [11]:
pulse_phase = {'pulse_name': 'Phase', 
               'upper_limit': 3.14, 
               'lower_limit': -3.14, 
               'bins_number': 101, 
               'time_name': 'time_p', 
               'amplitude_variation': 0.5, 
               'basis': {'basis_name': 'Fourier', 
                         'basis_class': 'Fourier', 
                         'basis_module': 'quocslib.pulses.basis.Fourier', 
                         'basis_vector_number': 5, 
                         'random_super_parameter_distribution': 
                         {'distribution_name': 'Uniform', 'distribution_class': 'Uniform', 
                          'distribution_module': 'quocslib.pulses.superparameter.Uniform', 
                          'lower_limit': 0.1, 'upper_limit': 3.0}}, 
               'scaling_function': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 1.0 + 0.0*t'}, 
               'initial_guess': {'function_type': 'lambda_function', 'lambda_function': 'lambda t: 0.0*t'}
              }

In [12]:
optimization_dictionary['pulses'] = [pulse_amplitude, pulse_phase]

### Put all together and get ready to start the optimization with Qudi-QuOCS

In [13]:
opti_comm_dict = {"optimization_dictionary": optimization_dictionary}

Load the optimization algorithm into the optimization logic and display it into the GUI

In [14]:
optimizationlogic.load_opti_comm_dict(opti_comm_dict)

#### Important: If the GUI is not showing the optimization dictionary, restart the Kernel

Print the optimization dictionary also here

**CHANGE** : We have to provide the standard deviation to the dCRAB algorithm. 

To do so we use the function:

``` fomlogic.update_fom(fom, std, status_code=0)```

after the measurement. **std** stays for the experimental standard deviation.

In [16]:
######################################################################################################
# Parameters and Settings
######################################################################################################

# This section is devoted to the initialization in the pulsed logic and optimization logic of the main
# settings and parameters to be usde in the creation ofthe pulse sequence and the optimization
# Iteration, controls and figure of merit to compare with QuOCS
# Just an example for debug
args_dict = {"is_noisy": True}
qubit = OneQubit2Fields(args_dict)

######################################################################################################
# Measurement
######################################################################################################
optimalcontrol.start_optimization()

# Just a time to check for latent time
last_time_fom = time.time()
# repeat the whole process until its manually stopped or QuOCS finsihed the optimization
# Wait few seconds before starting to get and return data
while not optimizationlogic.handle_exit_obj.is_user_running:
    time.sleep(0.1)
    if (time.time() - last_time_fom) > 30:
        print("Problem at the beginning! Surpassed the 30 secs")
        break

# print("Check before the loop starts: {0}".format(optimizationlogic.handle_exit_obj.is_user_running))
while  optimizationlogic.handle_exit_obj.is_user_running == True:
    # wait until QuOCS optimizes the controls
    # print("Wait until the controls logic gives the controls")
    while not controlslogic.are_pulses_calculated:
        time.sleep(0.1)
        # If the waiting time exceed 10 seconds left stop the optimization
        if time.time() - last_time_fom > 20:
            print("Too much time... Exit!")
            optimizationlogic.handle_exit_obj.is_user_running = False
            break
            

    #######################################################################################################
    # Get the Controls
    #######################################################################################################
    # Change the status of control calculations to avoid to evaluate the fom twice with the same controls
    controlslogic.are_pulses_calculated = False
    # Get the controls from the controls logic
    pulses, parameters, timegrids = controlslogic.pulses, controlslogic.parameters, controlslogic.timegrids
    #######################################################################################################
    # Perform the measurement
    #######################################################################################################
    #
    #
    #######################################################################################################
    # Calculate the figure of merit and the standard deviation
    #######################################################################################################
    # calculate the Figure of Merit
    fom_dict = qubit.get_FoM(pulses, timegrids, parameters)
    # Extract the fom and std
    fom, std = fom_dict["FoM"], fom_dict["std"]
    #######################################################################################################
    # Return the figure of merit
    #######################################################################################################
    # Update the figure of merit and the standard deviation to the fom logic
    fomlogic.update_fom(fom, std, status_code=0)
    # update the last time the fom is calculated
    last_time_fom = time.time()
    #######################################################################################################
    # Optional part
    #######################################################################################################
    # Print the data just for debug purpose
    print("FoM: {fom}, Std: {std}, status_code: {status_code}".format(fom=fom, std=std, status_code=0))
print("Optimization finished")

Is noise? : True
FoM: 0.18560703087418995, status_code: 0
FoM: 0.198837012729543, status_code: 0
FoM: 0.013280362532424768, status_code: 0
FoM: 0.05962755868502375, status_code: 0
FoM: 0.04053639731494435, status_code: 0
FoM: 0.02283593636622111, status_code: 0
FoM: 0.09640406588467475, status_code: 0
FoM: 0.19672320312872627, status_code: 0
FoM: 0.33668560296872757, status_code: 0
FoM: 0.02941567335584987, status_code: 0
FoM: 0.06683340714509145, status_code: 0
FoM: 0.3797351459300421, status_code: 0
FoM: 0.13160930855996175, status_code: 0
FoM: 0.15136242360914784, status_code: 0
FoM: 0.29896288688178735, status_code: 0
FoM: 0.14204822634564654, status_code: 0
FoM: 0.3090784977364705, status_code: 0
FoM: 0.13056904241521758, status_code: 0
FoM: 0.5181745595406889, status_code: 0
FoM: 0.278601314021155, status_code: 0
FoM: 0.5805296939561075, status_code: 0
FoM: 0.4002889635572872, status_code: 0
FoM: 0.1449660516035291, status_code: 0
FoM: 0.4488201397415466, status_code: 0
FoM: 0.26

In [ ]:
# Access to the optimizer object to get info about the optimization
optimizer_obj = optimizationlogic.optimizer_obj

In [ ]:
optimizer_obj

In [ ]:
# Best fom with std
print("FoM: {fom} +- {std}".format(fom=optimizer_obj.best_fom, std=optimizer_obj.std) 

In [ ]:
# Best controls
pulses_list, time_grids_list, parameters_list = optimizer_obj.get_best_controls()
t_amplitude = time_grids_list[0]
amplitude = pulses_list[0]
t_phase = time_grids_list[1]
phase = pulses_list[1]